In [1]:
%cd ../../..

/Users/behzadhaki/Github/GrooveTransformer


In [2]:
import os

import numpy as np
import pandas as pd

from bokeh.plotting import figure, show, save
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [ ]:
import matplotlib.pyplot as plt
from IPython.display import Audio

In [ ]:
from data import load_gmd_hvo_sequences

train_set = load_gmd_hvo_sequences(
    dataset_setting_json_path = "data/dataset_json_settings/4_4_Beats_gmd.json", 
    subset_tag = "train", 
    force_regenerate=False)

_1bar_set = []

# load with 16 steps only
for ix, exp_hvo_seq in enumerate(train_set):
    temp = exp_hvo_seq.copy_empty()
    if (exp_hvo_seq.hvo[16:, :].shape[0]>0):
        temp.hvo = exp_hvo_seq.hvo[16:, :]
        # temp.offsets = np.zeros(temp.offsets.shape)
        _1bar_set.append(temp)
    

In [ ]:
sample_ix = 100
_1bar_set[sample_ix].to_html_plot(show_figure=True)
_1bar_set[sample_ix].get_per_step_list_of_active_voices(use_label_instead_of_midi_number=False, print_mapping=True)

In [ ]:
'''
mask_creation_mode:
    [0] remove with higher probability proportional to inverse of the metrical level
    [1] remove with higher probability proportional to the metrical level
    [2] remove randomly
'''
sample_ix = 10
masked_version = _1bar_set[sample_ix].get_masked_variation(ratio_of_hits_to_mask = 0.8,
                                                           mask_creation_mode=1)
_1bar_set[sample_ix].to_html_plot(show_figure=True)
masked_version.to_html_plot(show_figure=True)

Audio(data=_1bar_set[sample_ix].synthesize(sf_path="hvo_sequence/soundfonts/Standard_Drum_Kit.sf2"), rate=44100)

In [ ]:
Audio(data=masked_version.synthesize(sf_path="hvo_sequence/soundfonts/Standard_Drum_Kit.sf2"), rate=44100)


In [65]:
Audio(data=masked_version.synthesize(sf_path="hvo_sequence/soundfonts/Standard_Drum_Kit.sf2"), rate=44100)


fluidsynth: error: Unknown integer parameter 'synth.sample-rate'


******
## Masking Strategies

### 1. Creating non-uniform masking probabilites based on the metrical profile
Here we mask a pattern based on the inverse of the metrical salience. That is, the more salient locations are less likely to be removed!

In [ ]:
# masking strategies
from hvo_sequence.metrical_profiles import Longuet_Higgins_METRICAL_PROFILE_4_4_16th_NOTE as metrical_profile
metrical_profile = metrical_profile[:16]
num_voices = 9

In [ ]:
inverse_ratios = np.array([1./salience for salience in  metrical_profile][:16])


fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 3))

axes[0].set_title("metrical_profile")
axes[0].set_xlabel("")
axes[0].set_xticks([0, 4, 8, 12, ])
axes[0].plot(range(len(metrical_profile[:16])), metrical_profile[:16])

axes[1].set_title("inverse ratios")
axes[1].set_xlabel("")
axes[1].set_xticks([0, 4, 8, 12, ])
axes[1].plot(range(len(inverse_ratios)), inverse_ratios)
plt.show()

In [ ]:
# grab an example
example = _1bar_set[sample_ix]
example.hits [:, 0]

In [ ]:
# Get masking probability at each hit location
# either use metric_profile or inverse_of_metric 

remove_less_salient_events_first = True

if remove_less_salient_events_first:
    masking_probs = np.transpose(np.transpose(example.hits) * inverse_ratios)
else:
    masking_probs = np.transpose(np.transpose(example.hits) * metrical_profile)
        
masking_probs = masking_probs / np.sum(masking_probs) 

In [ ]:
ratio_of_hits_to_mask = .1

total_hits = example.hits.sum()
num_hits_to_mask = np.ceil(total_hits * ratio_of_hits_to_mask)
print(num_hits_to_mask, total_hits)
while(True):
    mask = np.random.multinomial(num_hits_to_mask, masking_probs.flatten())
    if np.all(mask<=1):
        break
    
mask = mask.reshape(example.hits.shape)

removed_version = example.copy_zero()
removed_version.hvo = example.hvo - example.hvo * np.hstack((mask, mask, mask))


example.to_html_plot(show_figure=True)

removed_version.to_html_plot(show_figure=True)
